In [ ]:
import requests
from bs4 import BeautifulSoup as BS
import re
import pandas as pd
from time import sleep # this is to create a time delay

# 1. Navigate to https://www.billboard.com/charts/hot-100/. Using BeautifulSoup, extract out the This Week, artist, song, Last Week, Peak Position, and Weeks on Chart values into a pandas DataFrame. Hint: The HTML for the number one ranked song is slightly different from that of the rest of the songs.

In [ ]:
URL = 'https://www.billboard.com/charts/hot-100/'

response = requests.get(URL)
response

In [ ]:
soup = BS(response.text)
#soup

In [ ]:
soup.find('h3').text

In [ ]:
songs = soup.findAll('h3')
titles = []
for song in songs:
    meta_data = str(song)
    #print(meta_data)
    
    pattern = re.compile('a-no-trucate')
    
    if pattern.search(meta_data):
        song_cleaned = re.subn('[\n\t]', '', song.text)[0]
        #print(f'Result: "{song_cleaned}" is a song!\n')
        titles.append(re.subn('[\n\t]', '', song.text)[0])
    #else:
    #    print('Result: It is not a song\n')

In [ ]:
titles

In [ ]:
len(titles)

### Now to collect the other categories

In [ ]:
spans = soup.findAll('span')

In [ ]:
print(len(spans))
spans

In [ ]:
pattern = re.compile('lrv-u-padding-tb-050@mobile-max')
span_filter = []

for span in spans:
    span_text = str(span)
    #print(span_text,'\nNEW ENTRY\n')
    if pattern.search(span_text):
        span_filter.append(span)

In [ ]:
span_filter

In [ ]:
len(span_filter)

There are 3 entries of data for each song that we are not using. As such, I'll use a loop to filter which data to use and where we'll put it.

In [ ]:
i = 1
last_week = []
peak_pos = []
weeks_on_chart = []


for span in span_filter:
    if i%6 == 1:
        last_week.append(span.text.strip('\n\t'))
    elif i%6 == 2:
        peak_pos.append(span.text.strip('\n\t'))
        print()
    elif i%6 == 3:
        weeks_on_chart.append(span.text.strip('\n\t'))

    i += 1
#end

### Let me explain this loop

When I was cycling through the instances of the elements found in span_filter, I noticed that every 4th, 5th, and 6th entry was not visible on the website, meaning I had to disregard it. What this function is doing is just that: disregarding every 4th, 5th, and 6th element.

This is done using %. This is commonly referred to as "modulus" to math people like myself, but don't worry if that term scares you. The concept is very easy: it gives you remainders when you divide.

For instance, 5/2 = 2.5, but 5%2 = 1. This is because 2 goes into 5 a total of 2 times, but has a remainder of 1. Since the remainder is 1, that is what % returns. Weird right? Who would've thought that dividing with remainders would be used anywhere outside of elementary school?

Anyway, since there are 3 data points I need and 3 that I don't, I created an iterable variable (which is 'i') to keep track of which data point I was on. The reason I use i%6 is beacuse each song has 6 data points (again, with 3 that I need and 3 that I need to disregard). As such, I was able to succesfully filter out the ones I didn't need and assign the ones I did need.

In [ ]:
print(len(last_week))
last_week

In [ ]:
print(len(peak_pos))
peak_pos

In [ ]:
print(len(weeks_on_chart))
weeks_on_chart

### Creating the Data Frame to add to later

In [ ]:
songs_df = pd.DataFrame(titles)
songs_df.columns = ['Title']
songs_df

### Finding the artists for each song

In [ ]:
pattern = re.compile('a-no-trucate')
artists = []

for span in spans:
    span_text = str(span)
    #print(span_text,'\nNEW ENTRY\n')
    if pattern.search(span_text):
        artists.append(span.text.strip('\n\t'))

In [ ]:
artists

### Finding rankings of songs

In [ ]:
pattern = re.compile('u-letter-spacing-0080@tablet')
ranks = []

for span in spans:
    span_text = str(span)
    #print(span_text,'\nNEW ENTRY\n')
    if pattern.search(span_text):
        ranks.append(span.text.strip('\n\t'))

In [ ]:
ranks

In [ ]:
songs_df['Rank'] = ranks
songs_df = songs_df.set_index('Rank')
songs_df

In [ ]:
songs_df['Artist(s)'] = artists
songs_df['Last Week'] = last_week
songs_df['Peak Position'] = peak_pos
songs_df['Weeks on Chart'] = weeks_on_chart
songs_df

# 2. After getting the code working for the current chart, navigate to last week's chart. Notice how the url for the page changes. Write a function which will, given a date, return a pandas DataFrame containing the Billboard chart data for that date.

The url changes based on whatever was the last day of the week (for instance, 2023-11-04 was a Saturday, so that date represents the entire week). So what I need to do is, given a date, determine if it is Saturday, and if not, how to find the Saturday of the same week. This could be a case statement (if day_name == 'Tuesday': day = day+4), although it does seem to be inefficient. Oh well, I'm gonna try that anyway.

In [ ]:
import datetime as DateTime

In [ ]:
date_1 = DateTime.datetime.strptime(day_str, "%Y-%m-%d")

end_date = date_1 + DateTime.timedelta(days=1)
print(end_date.date())

In [ ]:
#def get_hot_100(day): ### day must be in format YYYY-MM-DD
#    date = DateTime.datetime.strptime(day, '%Y-%m-%d')

#    day_num = date.weekday()
#    print(day_num)
    
#    if day_num != 5:
#        if day_num < 5:
#            day_delta = 5 - date.weekday()
#            date = date + DateTime.timedelta(days=day_delta)
#        else:
#            date = date + DateTime.timedelta(days=6)
#    print(date.date())

In [ ]:
#get_hot_100('2023-11-05')

In [ ]:
some_day = DateTime.datetime.strptime('2023-11-13', '%Y-%m-%d')
some_day.weekday()

## After all of this hard work, I found out that you can just put the date in the URL. I'm so sad.

In [ ]:
def get_hot_100(date): # date must be in format YYYY-MM-DD
    date = str(date) # If it isn't already
    
    URL = 'https://www.billboard.com/charts/hot-100/' + date
    
    response = requests.get(URL)
    soup = BS(response.text)
    
    ### Titles
    songs = soup.findAll('h3')
    titles = []

    for song in songs:
        meta_data = str(song)

        pattern = re.compile('a-no-trucate')

        if pattern.search(meta_data):
            song_cleaned = re.subn('[\n\t]', '', song.text)[0]
            titles.append(re.subn('[\n\t]', '', song.text)[0])
    #end
    
    ### Defining spans
    spans = soup.findAll('span')
    
    pattern = re.compile('lrv-u-padding-tb-050@mobile-max')
    span_filter = []

    for span in spans:
        span_text = str(span)
        
        if pattern.search(span_text):
            span_filter.append(span)
    #end
    
    ### The Modulus Loop
    i = 1
    last_week = []
    peak_pos = []
    weeks_on_chart = []

    for span in span_filter:
        if i%6 == 1:
            last_week.append(span.text.strip('\n\t'))
        elif i%6 == 2:
            peak_pos.append(span.text.strip('\n\t'))
        elif i%6 == 3:
            weeks_on_chart.append(span.text.strip('\n\t'))

        i += 1
    #end
    
    ### Artists
    pattern = re.compile('a-no-trucate')
    artists = []

    for span in spans:
        span_text = str(span)

        if pattern.search(span_text):
            artists.append(span.text.strip('\n\t'))
    #end
    
    ### Ranks
    pattern = re.compile('u-letter-spacing-0080@tablet')
    ranks = []

    for span in spans:
        span_text = str(span)
    
        if pattern.search(span_text):
            ranks.append(span.text.strip('\n\t'))
    #end  
    
    ### Creating the Data Frame
    songs_df = pd.DataFrame(titles)
    songs_df.columns = ['Title']
    songs_df['Rank'] = ranks
    songs_df = songs_df.set_index('Rank')
    songs_df['Artist(s)'] = artists
    songs_df['Last Week'] = last_week
    songs_df['Peak Position'] = peak_pos
    songs_df['Weeks on Chart'] = weeks_on_chart
    
    return songs_df

# 3. Write a loop to retrieve the Billboard chart data for the last 10 weeks.

In [ ]:
today = DateTime.datetime.today()

for i in range(1,11): # range is left exclusive, so we use 11 to include 10
    #print(i)
    date = today - DateTime.timedelta(weeks=i)
    print(date.date())
    
    dataframes = []
    
    song_df = get_hot_100(str(date.date()))
    dataframes.append(song_df)
#end

dataframes